In [9]:
import ROOT
import numpy as np
from src.histogram import Histogram1D

In [10]:
ROOT.TH1.SetDefaultSumw2()  # Sets weighted binning in all ROOT histograms by default
ROOT.gROOT.SetBatch()  # Prevents TCanvas popups

In [11]:
a = np.random.normal(size=1000)
w = np.random.random(1000)
nbins = 20
xmin = -1
xmax = 1

In [12]:
hH = Histogram1D(bins=(nbins, xmin, xmax))

In [13]:
%%timeit
hH.fill(a, weight=w)

8.97 µs ± 28.2 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [21]:
hH.view(flow=True).variance

array([4.27394177e+07, 4.00000000e+00, 7.97883213e+06, 7.41176394e+06,
       9.78752200e+06, 7.67534516e+06, 9.20558341e+06, 8.43393334e+06,
       6.84327631e+06, 1.11792446e+07, 7.30527227e+06, 1.12497601e+07,
       8.52544280e+06, 1.21891296e+07, 1.17583117e+07, 9.84540687e+06,
       1.38828840e+07, 8.17661672e+06, 8.80577638e+06, 8.68057862e+06,
       9.27361794e+06, 3.35382373e+07])

In [4]:
hR = ROOT.TH1F('test', 'test;x;entries', nbins, xmin, xmax)
hR

In [7]:
%%timeit
for i, j in zip(a, w):
	hR.Fill(i, j)

277 µs ± 16.9 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [27]:
edges = [hR.GetBinLowEdge(i + 1) for i in range(hR.GetNbinsX())]
edges.append(edges[-1] + hR.GetBinWidth(hR.GetNbinsX()))
print(edges)

[-1.0, -0.9, -0.8, -0.7, -0.6, -0.5, -0.3999999999999999, -0.29999999999999993, -0.19999999999999996, -0.09999999999999998, 0.0, 0.10000000000000009, 0.20000000000000018, 0.30000000000000004, 0.40000000000000013, 0.5, 0.6000000000000001, 0.7000000000000002, 0.8, 0.9000000000000001, 1.0000000000000002]


In [32]:
print(hH.integral)

256643765.9079706


In [8]:
[hR.GetBinContent(i + 1) for i in range(hR.GetNbinsX())]

[114711.8671875,
 99741.6953125,
 69407.3359375,
 113014.6015625,
 151185.65625,
 182838.890625,
 155302.265625,
 209960.21875,
 165627.953125,
 155813.171875,
 128193.7109375,
 173992.6875,
 223634.90625,
 186566.171875,
 179495.546875,
 157899.828125,
 75775.515625,
 140699.15625,
 91036.03125,
 102935.0703125]

In [28]:
%%timeit
hH.fill(a, weight=w)

8.55 µs ± 42.1 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [5]:
%%timeit
map(hR.Fill, *zip(a, w))

70.7 µs ± 458 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [13]:
from multiprocessing.pool import ThreadPool as Pool
import os

In [14]:
%%timeit
with Pool(os.cpu_count()) as p:
	p.map(hR.Fill, a)

2.55 ms ± 259 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
%%timeit
{hR.Fill(i) for i in a}

In [22]:
import time

boost_times = []
root_loop_times = []
root_map_times = []
root_comp_times = []
root_pool_times = []
x = np.logspace(10, 10**7, 7)
for n in x:
	print("n = ", n)
	a = np.random.normal(size=n)
	w = np.random.random(n)

	hR = ROOT.TH1F('test', 'test;x;entries', nbins, xmin, xmax)
	hH = Histogram1D(bins=(nbins, xmin, xmax))

	t1 = time.time()
	hH.fill(a, weight=w, threads=0)
	boost_times.append(time.time() - t1)

	t2 = time.time()
	for ia, iw in zip(a, w):
		hR.Fill(ia, iw)
	root_loop_times.append(time.time() - t2)

	t3 = time.time()
	map(hR.Fill, *zip(a, w))
	root_map_times.append(time.time() - t3)

	t4 = time.time()
	{hR.Fill(ia, iw) for ia, iw in zip(a, w)}
	root_comp_times.append(time.time() - t4)

	t5 = time.time()
	with Pool(os.cpu_count()) as p:
		p.map(hR.Fill, a)
	root_pool_times.append(time.time() - t5)

import matplotlib.pyplot as plt
plt.plot(boost_times, x, label='boost-histogram')
plt.plot(root_loop_times, x, label='ROOT - for loop')
plt.plot(root_map_times, x, label='ROOT - map')
plt.plot(root_comp_times, x, label='ROOT - comprehension')
plt.plot(root_pool_times, x, label='ROOT - thread pool')
plt.legend()
plt.xlabel('array length')
plt.ylabel('time to fill')
plt.show()


IndentationError: unindent does not match any outer indentation level (<tokenize>, line 31)